In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import os
import tabula
import re
import PyPDF2
from word2numberi18n import w2n
import locale
import io
import glob
import sys
from PIL import Image
import pytesseract
from pdf2image import convert_from_path
import gc


os.environ['w2n.lang'] = 'es'
locale.setlocale(locale.LC_TIME, 'es_ES')

def fill_text_field(driver, xpath, text):
    # Funcion que llena texto en un campo de la pagina
    text_input = driver.find_element(By.XPATH, xpath)
    text_input.clear()
    text_input.send_keys(text)

def select_option(driver, xpath, index):
    # Selecciona una opcion en un menu desplegable
    select_object = Select(driver.find_element(By.XPATH, xpath))
    select_object.select_by_value(index)


url = 'https://www.pjud.cl/tribunales/corte-suprema'
chromeOptions = webdriver.ChromeOptions()
path = os.path.join(os.getcwd(), "output2\\")
prefs = {"download.default_directory" : path,  "directory_upgrade": True}
chromeOptions.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options = chromeOptions)
driver.get(url)
driver.implicitly_wait(10)

#años = list(range(2020, 2022))
años = [2015]
meses = ["04"]

for año in años:
    # Estos indices se obtienen de las opciones de la pagina, cada uno es un mes, cambiar para obtener mas meses
    #for index in range(1, 2):
        # Se envia la info a la pagina
    select_option(driver, '/html/body/div[11]/div/div/div[2]/form/div[2]/div[1]/div/select', str(año))
    for mes in meses:
        select_option(driver, '/html/body/div[11]/div/div/div[2]/form/div[2]/div[2]/div/select', mes)
        select = Select(driver.find_element(By.XPATH, "/html/body/div[11]/div/div/div[2]/form/div[2]/div[3]/div/select")) #get all the options into a list
        optionsList = []
        for item in select.options:
            optionsList.append(item.get_attribute("value"))            
            for optionValue in optionsList:
                select_option(driver, '/html/body/div[11]/div/div/div[2]/form/div[2]/div[3]/div/select', optionValue)

        # Se hace click con los datos llenados
                driver.find_element(By.XPATH, '/html/body/div[11]/div/div/div[2]/form/div[2]/div[4]/button').click()
        
        # Se descargan los datos
                wait = WebDriverWait(driver, 10) 
    #descarga por salas cambia último tr: 1, 2, 3, 4 - no siempre está disponible, el mismo día, las mismas salas
                try: 
                    boton_descarga = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[11]/div/div/div[2]/table/tbody/tr[1]/td[1]/a')))
                    boton_descarga.click()
                except: 
                    pass
                try: 
                    boton_descarga = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[11]/div/div/div[2]/table/tbody/tr[2]/td[1]/a')))
                    boton_descarga.click()
                except: 
                    pass
                try: 
                    boton_descarga = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[11]/div/div/div[2]/table/tbody/tr[3]/td[1]/a')))
                    boton_descarga.click()
                except: 
                    pass
                try: 
                    boton_descarga = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[11]/div/div/div[2]/table/tbody/tr[4]/td[1]/a')))
                    boton_descarga.click()
                except: 
                    pass
driver.close


pytesseract.pytesseract.tesseract_cmd = "C:/Program Files (x86)/Tesseract-OCR/tesseract.exe"


palabras = ['licencia', 'comisión', 'permiso', 'feriado', 'inhabilidad', 'subroga', 'vacancia']



pdf_files = glob.glob(os.path.join("C:/Users/jwile/WebScraping/WebScraping-Cs/scan",'*.pdf'))


columns = ['Fecha', 'Sala', 'Presidente', 'Ministros', 'Integrantes', 'Causa']
df = pd.DataFrame(columns = columns)
pages = []


for pdf_file in pdf_files[0:10]:
    print(pdf_file)
    imgs = convert_from_path(pdf_file, 300)
    Ministros = []
    Integrantes = []
    Causa = []
    for page in imgs:
        page.save('out.jpg', 'JPEG')
        text = pytesseract.image_to_string(page, lang ='spa')
        if re.search("INSTALACION", text):
            Presidente = re.findall('(?<=PRESIDENTE:).*?(?=\\\n)', text)
            Ministros.append(re.findall('(?<=MINISTROS:).*?(?=\\\n)', text))
            Ministros.append(re.findall('.*?(?=\sy\sEL\sABOGADO)', text))
            Integrantes.append(re.findall('(?<=INTEGRANTES:).*?(?=\\\n)', text))
            Integrantes.append(re.findall('(?<=INTEGRANTE:).*?(?=\\\n)', text))
            if re.findall('(?<=En Santiago,).*?(?=,)', text) != []:
                Fecha = re.findall('(?<=En Santiago,).*?(?=,)', text) 
            elif re.findall('(?<=En Sanfiago,).*?(?=,)', text) != []:
                Fecha = re.findall('(?<=En Sanfiago,).*?(?=,)', text) 
            else: Fecha = []
            try:
                Sala = re.findall('(?<=\\s).*?(?=\\sSALA)', text)[0]
            except:
                pass
            for palabra in palabras:         
                if palabra in text:
                    Causa.append(palabra)
            df = df.append(
               {"Fecha" : str(Fecha), "Sala" : str(Sala), 'Presidente' : Presidente, 'Ministros' : Ministros,
                "Integrantes" : Integrantes, 'Causa' : Causa},
               ignore_index=True,
            )
df
df2 = df.copy()
        
        
        



print(df)

df3 = df2.copy()

sum(df3['Fecha'].isna())

df3['Fecha']
df3['Fecha'].value_counts()

df3['Fecha'] = df3['Fecha'].str[2:]

df3['Fecha'] = df3['Fecha'].str[:-2]

df3['Fecha'].value_counts()

df3['Fecha'] = df3['Fecha'].str.replace("treinta y uno", "treintaiuno")
df3['Fecha'] = df3['Fecha'].str.replace('\ba\b','')

df3['Fecha'] =  df3['Fecha'].str.replace(r"\b[a-zA-Z]\b", "")
df3['Fecha'] = df3['Fecha'].str.lstrip()
df3['Fecha']




df3['Dia'] = df3['Fecha'].str.split().str.get(0)
df3["Ano"] = df3["Fecha"].str.extract(r'(\w+)$')
df3['Dia'] = df3['Dia'].str.replace("treintaiuno", "treinta y uno")

nums = {'^uno$': '1', '^dos$': '2', '^tres$': '3', "^cuatro$":'4', "^cinco$" : '5', "^seis$":'6', "^siete$": '7', 
        '^ocho$':'8',
       '^nueve$':'9', '^mueve$':'9', 'diez':'10', 'once':'11', 'doce':'12', 'trece':'13','catorce':'14', 'quince': '15', 'quinee': '15',
        'dieciséis':'16',
        'diecisiete':'17', 'dieciocho':'18', 'diecinueve':'19', 'veinte':'20', 'veintiuno':'21','veintidós':'22', 
        'veintidos':'22', 'veintifiete' : '27', 'veinfisiete': '27', 'veíntisiete' : '27',
        'veintitrés':'23', 'veinticuatro':'24', 'veinticinco':'25', 'veintiséis':'26', 'veintisiete':'27',
        'veintiocho':'28', 'veinfiocho': '28','veintinueve': '29', 'veinfinueve': '29', '^treinta$': '30', 'tremía' : '30',
        'treinta y uno': '31'}
        

for old, new in nums.items():
    df3['Dia'] = df3['Dia'].str.replace(old, new, regex=True)

df3['Dia'].value_counts()

nums = {'^veinte$': '20', '^veintiuno$': '21', '^veintidós$': '22', '^quince$' : '15', 'quinee' :'15', 'nan': '15',
       'quínce': '15', 'quimce':'15', 'quínee' : '15'}
        

for old, new in nums.items():
    df3['Ano'] = df3['Ano'].str.replace(old, new, regex=True)

df3['Ano'].value_counts()

#convierte a string para juntar
df3['Ano'] = df3['Ano'].astype(str)
df3['Dia'] = df3['Dia'].astype(str)#aisla y convierte a una columna mes
df3["Mes"] = df3["Fecha"]
df3['Mes'] = df3['Mes'].str.split(n=1).str[1]
df3['Mes'] = df3['Mes'].str.lstrip()
df3['Mes'] = df3['Mes'].str.split(' ').str[1] 

df3['Mes'].value_counts()
meses = {'abrij': 'abil', 'abríl': 'abril', 'de': 'abril', 'abil' : 'abril'}
        

for old, new in meses.items():
    df3['Mes'] = df3['Mes'].str.replace(old, new, regex=True)

df3['Mes'].value_counts()

#concatenación fecha en formato
df3['Fecha2'] = df3['Dia'] + '/' + df3['Mes'] + '/' + df3['Ano']
#transformación a datetime
df3['Fecha2'] = pd.to_datetime(df3["Fecha2"], format = '%d/%B/%y')

df3

df4 = df3.copy()

df4

df4['Presidente'] = df4['Presidente'].str[0]


df4['Presidente'] = df4['Presidente'].str.replace('SEÑOR','')
df4['Presidente'] = df4['Presidente'].str.replace('SEÑORA','')
df4['Presidente'] = df4['Presidente'].str.replace('.','')
df4['Presidente'].value_counts()

df4['Ministros'] = df3['Ministros'].copy()

df4['Ministros2'] = df4['Ministros'].str[1]
df4['Ministros2'] = df4['Ministros2'].str[0]
df4 = df4.fillna('')
df4['Ministros'] = df4['Ministros'].str[0]
df4['Ministros'] = df4['Ministros'].str[0]

df4['Ministros'] = df4['Ministros'] + ' ' + df4['Ministros2']

df4['Ministros']

df4['Ministros'] = df4['Ministros'].str.upper()

df4['Ministros'] = df4['Ministros'].str.replace('SEÑORES','')
df4['Ministros'] = df4['Ministros'].str.replace('SEÑORA','')
df4['Ministros'] = df4['Ministros'].str.replace('SEÑOR','')
df4['Ministros'] = df4['Ministros'].str.replace('MA GGI','MAGGI')
df4['Ministros'] = df4['Ministros'].str.replace('MAGGL','MAGGI')
df4['Ministros'] = df4['Ministros'].str.replace('LOS ABOGADOS','')


df4['Ministros'] = df4['Ministros'].str.replace(r'\b\w\b','')


df4['Ministros'] = df4['Ministros'].str.replace(r'\bEL\b','')
df4['Ministros'] = df4['Ministros'].str.replace(r'\bSA\b','')
df4['Ministros'] = df4['Ministros'].str.replace(',','')

df4['Ministros']

df4['Integrantes2'] = df4['Integrantes'].str[1]
df4['Integrantes2'] = df4['Integrantes2'].str[0]

df4['Integrantes'] = df4['Integrantes'].str[0]
df4['Integrantes'] = df4['Integrantes'].str[0]

df4= df4.fillna('')

df4['Integrantes'] = df4['Integrantes'] + ' ' + df4['Integrantes2']

df4['Integrantes']

df4['Integrantes'] = df4['Integrantes'].str.upper()

df4['Integrantes'] =  df4['Integrantes'].str.replace('SEÑORES','')
df4['Integrantes'] = df4['Integrantes'].str.replace('SEÑORA','')
df4['Integrantes'] = df4['Integrantes'].str.replace('SEÑOR','')
df4['Integrantes'] = df4['Integrantes'].str.replace('.','')
df4['Integrantes'] = df4['Integrantes'].str.replace(r'\b\w\b','')


df4['Abogados'] = df4['Integrantes'].copy()

df4['Integrantes'] = df4['Ministros'] + '' + df4['Abogados']

df4['Integrantes']

df4['Integrantes'] = df4['Integrantes'].str.replace('   ',' ')
df4['Integrantes'] = df4['Integrantes'].str.replace('  ',' ')

df4['Integrantes'] = df4['Integrantes'].str.lstrip()

df4['Integrantes2'] = df4['Integrantes'].str.replace(" ", ",")


df4['Integrantes2'] = df4["Integrantes2"].str.split(',', n = 3, expand = False)

df4['Integrante1'] = df4['Integrantes2'].str[0]
df4['Integrante2'] = df4['Integrantes2'].str[1]
df4['Integrante3'] = df4['Integrantes2'].str[2]
df4['Integrante3'] = df4['Integrantes2'].str[4]

len(df4)

df5 = df4
df5 = df5.drop(['Fecha', 'Integrantes2', 'Dia', 'Ano', 'Mes'], axis = 1)


len(df5)

df5['Causa1'] = df3['Causa'].str[0]
df5['Causa2'] = df3['Causa'].str[1]
df5['Causa3'] = df3['Causa'].str[2]
df5['Causa4'] = df3['Causa'].str[3]

df5 = df5.drop(['Causa'], axis = 1)

df5 = df5.drop_duplicates()
len(df5)

df5.to_csv("Primer intento scan")